In [ ]:
!uv pip install guidance==0.2.4 SentencePiece==0.2.0 accelerate==1.9.0

In [ ]:
from huggingface_hub import login, whoami

# Connexion
login("hfXXX")

In [ ]:
from guidance import models, guidance

# Load a Transformers-based LLM
llm = models.Transformers(
    "mistralai/Mixtral-8x7B-Instruct-v0.1",
    device_map="cuda",
    torch_dtype="auto",               # Let it choose best dtype (e.g., float16)
)

In [ ]:
@guidance
def donnees_accords_heures_supp(lm,input_text):
    lm += f"""Le JSON suivant représente les données structurées des accords d'heures supplémentaires en entreprise.

    Text:
    {input_text}
    
    ```json
{{
    "index": "{gen('index', stop='\"')}",
    "base_legale_hebdomadaire": {gen('base_legale_hebdomadaire', regex='[0-9]+', stop=',')},
    "duree_annuel_heures": {gen('duree_annuel_heures', regex='[0-9]+', stop=',')},
    "contingent_annuel_heures_supplementaires": {gen('contingent_annuel_heures_supplementaires', regex='[0-9]+', stop=',')},
    "nombre_taux_majoration_differents": {gen('nombre_taux_majoration_differents', regex='[0-9]+', stop=',')},
    "premier_taux_majoration": {gen('premier_taux_majoration', regex='[0-9]+', stop=',')},
    "plage_premier_taux_majoration": "{gen('plage_premier_taux_majoration', stop='\"')}",
    "deuxieme_taux_majoration": {gen('deuxieme_taux_majoration', regex='[0-9]+', stop=',')},
    "plage_deuxieme_taux_majoration": "{gen('plage_deuxieme_taux_majoration', stop='\"')}",
    "troisieme_taux_majoration": {gen('troisieme_taux_majoration', regex='[0-9]+', stop=',')},
    "plage_troisieme_taux_majoration": "{gen('plage_troisieme_taux_majoration', stop='\"')}",
    "presence_repos_compensateur_remplacement": {select(options=['true', 'false'], name='presence_repos_compensateur_remplacement')},
    "taux_majoration_contrepartie_obligatoire_en_repos": {gen('taux_majoration_contrepartie_obligatoire_en_repos', regex='[0-9]+', stop=',')},
    "delai_prevenance": "{gen('delai_prevenance', stop='\"')}"
}}
```"""
    return lm

example_text = """
Le contingent annuel d’heures supplémentaires est fixé à 220 heures. Le premier taux de majoration est de 25% et s’applique de la 36e à la 43e heure.
"""

templated_fn = donnees_accords_heures_supp(
    input_text=example_text
)

result = templated_fn(model=llm)

print(result)